<a href="https://colab.research.google.com/github/nicohorn/labo2025v/blob/main/src/rpart/290_TareaHogar_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea para el Hogar 02

Esta Tarea para el Hogar 02 se entrega el final de la segunda clase
<br> se espera de usted que intente avanzar con los desafios propuestos y que los traiga terminados para la Clase 03, ya que se analizarán los resultados

##  1. Ensembles de Modelos

Vea el siguiente video [BBC - The Code - The Wisdom of the Crowd](https://www.youtube.com/watch?v=iOucwX7Z1HU)    ( 5 min)


Lea los siguientes artículos


*   [The Wisdom of Crowds (Vox Populi) by Francis Galton](https://www.all-about-psychology.com/the-wisdom-of-crowds.html)  (10 min)
*   [A Gentle Introduction to Ensemble Learning](https://machinelearningmastery.com/what-is-ensemble-learning/)  (10 min)





---



##  2.  Zero2Hero   primera parte
Se han lanzado los primeros fascículos coleccionables llamados "from Zero to Hero" que muy detalladamente, paso a paso enseñan todo lo necesario de R para entender los scripts oficiales de la asignatura.
Están en el repositorio oficial de la asignatura, carpeta  **src/zero2hero**



---



## 3.  Grid Search

Busque en internet el precido significado de los hiperparámetros de la librería **rpart**  que está implementando el algoritmo **CART**  Classification and Regression Trees  propuesto en el año 1984 por Leo Brieman:

*   cp
*   maxdepth
*   minsplit
*   minbucket

Entienda que valores es razonable tome cada hiperparámetro,  en particular profundice en el hiperparámetro  **cp**  y la posibilidad que tome valores negativos.  Es válido consultar a su amigo de *capacidades especiales*  ChatGPT


En las siguientes celdas a un notebook incompleto, un esqueleto de codigo brindado a modo de facilitarle la tarea de codeo y permitir que su valiosa cognición se concentre temas conceptuales de Ciencia de Datos

Modifiquelo agregando loops para que recorra TODOS los hiperparámetros de rpart  < cp, maxdepth, minsplit, minbucket >, y luego póngalo a correr. Recuerde cambiar por SU semilla
Tenga muy presente la granularidad que eligirá para cada hiperparámetro.

### Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



limpio el ambiente de R

In [28]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,831890,44.5,1454482,77.7,1454482,77.7
Vcells,1567549,12.0,94652197,722.2,108837180,830.4


In [29]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")
if (!require("primes")) install.packages("primes")
require("primes")

Aqui debe poner SU semiila primigenia

In [30]:
PARAM <- list()
# reemplazar por su primer semilla
PARAM$semilla_primigenia <- 153929
PARAM$qsemillas <- 1

PARAM$training_pct <- 70L  # entre  1L y 99L

# elegir SU dataset comentando/ descomentando
PARAM$dataset_nom <- "~/datasets/dataset_pequeno.csv"

In [31]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)
#   crea una particion 70, 30

particionar <- function(data, division, agrupa = "", campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}


In [32]:
ArbolEstimarGanancia <- function(semilla, training_pct, param_basicos) {
  # particiono estratificadamente el dataset
  particionar(dataset,
    division = c(training_pct, 100L -training_pct),
    agrupa = "clase_ternaria",
    seed = semilla # aqui se usa SU semilla
  )

  # genero el modelo
  # predecir clase_ternaria a partir del resto
  modelo <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_basicos
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion <- predict(modelo, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad

  # prediccion es una matriz con TRES columnas,
  #  llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  # cada columna es el vector de probabilidades


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test <- dataset[
    fold == 2,
    sum(ifelse(prediccion[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada <- ganancia_test / (( 100 - PARAM$training_pct ) / 100 )

  return(
    c( list("semilla" = semilla),
      param_basicos,
      list( "ganancia_test" = ganancia_test_normalizada )
     )
  )
}


In [33]:
ArbolesMontecarlo <- function(semillas, param_basicos) {

  # la funcion mcmapply  llama a la funcion ArbolEstimarGanancia
  #  tantas veces como valores tenga el vector  PARAM$semillas
  salida <- mcmapply(ArbolEstimarGanancia,
    semillas, # paso el vector de semillas
    MoreArgs = list(PARAM$training_pct, param_basicos), # aqui paso el segundo parametro
    SIMPLIFY = FALSE,
    mc.cores = detectCores()
  )

  return(salida)
}


In [34]:
# carpeta de trabajo
# por fabor cambiar numero de experimento si se cambia el loop principal
setwd("/content/buckets/b1/exp")
experimento <- "HT2900"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [35]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [36]:

# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, PARAM$qsemillas )


In [37]:
# genero la data.table donde van los resultados detallados del Grid Search
# un registro para cada combinacion de < semilla, parametros >

if(file.exists("gridsearch_detalle.txt")){
  tb_grid_search_detalle <- fread("gridsearch_detalle.txt")
}else{
  tb_grid_search_detalle <- data.table(
    semilla = integer(),
    cp = numeric(),
    maxdepth = integer(),
   minsplit = integer(),
    minbucket = integer(),
    ganancia_test = numeric()
  )
}

nrow( tb_grid_search_detalle )

[1] 0

Esta es la parte del código que usted debe expandir a TODOS los hiperparámetros de rpart,
<br>ya que actualmente apenas recorre  maxdepth y  minsplit  dejando fijos  cp=-0.5  y minbucket=5

In [38]:
# Reduje cantidad de valores para encontrar zonas prometedoras

iter <- 0
tiempo_inicio <- Sys.time()

# Valores REDUCIDOS para exploración
valores_cp <- c(-1, -0.5, 0, 0.01, 0.1)  # 7 -> 5 valores
valores_maxdepth <- c(4, 8, 12)  # 6 -> 3 valores
valores_minsplit <- c(1000, 400, 100, 20)  # 9 -> 4 valores
valores_minbucket <- c(5, 10, 30)  # 7 -> 3 valores

# Total: 5 × 3 × 4 × 3 = 180 combinaciones (vs 2646 combinaciones iniciales)
# Tiempo estimado: ~2 horas (vs 29 horas iniciales)

total_iteraciones <- length(valores_cp) * length(valores_maxdepth) *
                     length(valores_minsplit) * length(valores_minbucket)

cat("Total de combinaciones a explorar:", total_iteraciones, "\n")
cat("Semillas por combinación:", PARAM$qsemillas, "\n")
cat("Tiempo estimado: ~", round(total_iteraciones * 40 / 60, 1), "minutos\n\n")

for (vcp in valores_cp) {
  for (vmax_depth in valores_maxdepth) {
    for (vmin_split in valores_minsplit) {
      for (vmin_bucket in valores_minbucket) {

        iter <- iter + 1

        if(iter * PARAM$qsemillas < nrow(tb_grid_search_detalle) + 1) {
          cat("Iter", iter, "- Ya procesada\n")
          next
        }

        tiempo_iter_inicio <- Sys.time()

        param_basicos <- list(
          "cp" = vcp,
          "maxdepth" = vmax_depth,
          "minsplit" = vmin_split,
          "minbucket" = vmin_bucket
        )

        ganancias <- ArbolesMontecarlo(PARAM$semillas, param_basicos)

        tb_grid_search_detalle <- rbindlist(
          list(tb_grid_search_detalle,
               rbindlist(ganancias))
        )

        tiempo_iter_fin <- Sys.time()
        duracion_iter <- as.numeric(difftime(tiempo_iter_fin, tiempo_iter_inicio, units = "secs"))
        tiempo_transcurrido <- as.numeric(difftime(tiempo_iter_fin, tiempo_inicio, units = "mins"))
        tiempo_promedio_iter <- tiempo_transcurrido / iter
        iteraciones_restantes <- total_iteraciones - iter
        tiempo_estimado_restante <- tiempo_promedio_iter * iteraciones_restantes

        cat(sprintf("Iter %d/%d (%.1f%%) | cp=%.2f md=%d ms=%d mb=%d | ",
                    iter, total_iteraciones, (iter/total_iteraciones)*100,
                    vcp, vmax_depth, vmin_split, vmin_bucket))
        cat(sprintf("Dur: %.1fs | Restante: %.1fmin\n",
                    duracion_iter, tiempo_estimado_restante))
        flush.console()
      }
    }

    cat("\n>>> Guardando checkpoint en maxdepth =", vmax_depth, "<<<\n\n")
    fwrite(tb_grid_search_detalle,
           file = "gridsearch_detalle.txt",
           sep = "\t")
  }
}

tiempo_total <- as.numeric(difftime(Sys.time(), tiempo_inicio, units = "mins"))
cat("\n=================================================\n")
cat("Exploración completada\n")
cat("Tiempo total:", sprintf("%.1f minutos\n", tiempo_total))
cat("=================================================\n")


Total de combinaciones a explorar: 180 
Semillas por combinación: 1 
Tiempo estimado: ~ 120 minutos

Iter 1/180 (0.6%) | cp=-1.00 md=4 ms=1000 mb=5 | Dur: 38.2s | Restante: 114.3min
Iter 2/180 (1.1%) | cp=-1.00 md=4 ms=1000 mb=10 | Dur: 39.4s | Restante: 115.3min
Iter 3/180 (1.7%) | cp=-1.00 md=4 ms=1000 mb=30 | Dur: 38.5s | Restante: 114.3min
Iter 4/180 (2.2%) | cp=-1.00 md=4 ms=400 mb=5 | Dur: 38.9s | Restante: 113.8min
Iter 5/180 (2.8%) | cp=-1.00 md=4 ms=400 mb=10 | Dur: 39.2s | Restante: 113.4min
Iter 6/180 (3.3%) | cp=-1.00 md=4 ms=400 mb=30 | Dur: 39.4s | Restante: 113.0min
Iter 7/180 (3.9%) | cp=-1.00 md=4 ms=100 mb=5 | Dur: 41.3s | Restante: 113.3min
Iter 8/180 (4.4%) | cp=-1.00 md=4 ms=100 mb=10 | Dur: 39.1s | Restante: 112.6min
Iter 9/180 (5.0%) | cp=-1.00 md=4 ms=100 mb=30 | Dur: 38.2s | Restante: 111.6min
Iter 10/180 (5.6%) | cp=-1.00 md=4 ms=20 mb=5 | Dur: 41.3s | Restante: 111.6min
Iter 11/180 (6.1%) | cp=-1.00 md=4 ms=20 mb=10 | Dur: 38.1s | Restante: 110.6min
Iter 12/1

In [39]:
fwrite( tb_grid_search_detalle,
   file = "gridsearch_detalle.txt",
   sep = "\t"
)

In [40]:
# cantidad de registros de la tabla
nrow(tb_grid_search_detalle)

[1] 180

In [41]:
# muestro la tabla
tb_grid_search_detalle

semilla,cp,maxdepth,minsplit,minbucket,ganancia_test
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
644593,-1,4,1000,5,45980000
644593,-1,4,1000,10,45980000
644593,-1,4,1000,30,48850000
644593,-1,4,400,5,45110000
644593,-1,4,400,10,45110000
644593,-1,4,400,30,48940000
644593,-1,4,100,5,45110000
644593,-1,4,100,10,45110000
644593,-1,4,100,30,48940000


In [42]:
# genero y grabo el resumen
tb_grid_search <- tb_grid_search_detalle[,
  list( "ganancia_mean" = mean(ganancia_test),
    "qty" = .N ),
  list( cp, maxdepth, minsplit, minbucket )
]


In [43]:
# ordeno descendente por ganancia
setorder( tb_grid_search, -ganancia_mean )


In [44]:
# veo los 10 mejores hiperparámetros
tb_grid_search[1:10]

cp,maxdepth,minsplit,minbucket,ganancia_mean,qty
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
-1.0,12,1000,10,52250000,1
-0.5,12,1000,10,52250000,1
-1.0,8,1000,10,50780000,1
-0.5,8,1000,10,50780000,1
-1.0,8,1000,5,50050000,1
-0.5,8,1000,5,50050000,1
-1.0,12,1000,5,49300000,1
-0.5,12,1000,5,49300000,1
-1.0,4,20,30,49030000,1


In [45]:
# genero un id a la tabla
tb_grid_search[, id := .I ]

fwrite( tb_grid_search,
  file = "gridsearch.txt",
  sep = "\t"
)


# 4.  Análisis de resultados de Grid Search

La salida de la corrida anterior queda en ~/buckets/b1/exp/HT2900  que corresponde a su Google Drive
<br>HT significa Hyperparameter Tuning
<br>El Grid Search es un método de fuerza bruta de un altísimo costo computacional.
<br>Queremos ver si es posible crear un algoritmo de optimización de hiperparámetros que se ahorre recorrer ciertas porciones muy malas del espacio de búsqueda. Algo del estilo “cada vez que pruebo una combinación de hiperparámetros donde  cp > 1 , la ganancia es muy mala, con lo cual ni vale la pena perder el tiempo explorando en esa region”


<br>Levante el archivo de salida gridsearch.txt  a una planilla tipo Excel y analícelo detenidamente
<br>Ordene por ganancia_mean descendente
<br>
<br>El de mayor ganancia_mean  decimos que es el primero del ranking
En Zulip, correspondiente channel  #Tarea Hogar 02 , topic Analisis Grid Search   intente contestar estas preguntas:

* ¿Qué combinaciones de hiperparámetros poseen una ganancia muy buena?
* ¿Hay algun hiperparámetro que para cierto valor siempre genera una ganancia muy mala, a independientemente de lo que valgan los otros hiperparámetros ?
* ¿Que combinaciones de hiperparámetros es pésima y hubiera sido bueno ahorrarse esas corridas ?

( tiempo estimado 30 minutos, dificultad media )